# Tutorial for ibahd-api Python package

1. Introduction
2. Setup
    - 2.1 Activate ibaHD-API
    - 2.2 Setting up the example HD store
    - 2.3 Installation of the ibahd-api Python package
    - 2.4 Test for successfull installation
3. Application of time_periods
    - 3.1 Parametrization for connection with ibaHD-Server
    - 3.2 Create sample client and display available HD stores
        - 3.2.1 Initialization of ibaHD-API client
        - 3.2.2 Exemplified function calls

## 1. Introduction
In the first part of this tutorial, the user gets introduced to the ibaHD-API in general. Then, this notebook exemplifies the functions of the ibahd-api Python package which can be used to get process data from an ibaHD-Server into a Python script. Some helper functions were added for better and easier usability.

### gRPC Fundamentals
ibaHD-API uses the modern open source framework gRPC (Google Remote Procedure Call). On their website the following [overview](https://grpc.io/docs/what-is-grpc/introduction/#overview) is provided to show its basic functionality:

<p align="left"> <img width=500 style="padding-left: 100px" src="../assets/gRPC_server_client.svg"> </p>

gRPC is based on the definition of a service and remotely callable methods including their parameters and return types. The server side (the ibaHD-Server in our case) implements the interface and runs a gRPC server to handle client calls. The client side runs a stub (also referred as client) that provides the same methods as the server.

gRPC uses ProtoBuf (Protocol Buffers, *.proto* extension) for serializing structured data like different request and response messages. After defining a proto file the ProtoBuf compiler *protoc* is used to generate data access classes in the desired language. These classes are utilized in the application to populate, serialize and retrive the sent messages. More information on Protocol Buffers can be found at [ProtoBuf](https://protobuf.dev/overview/).

A [Quick Start Guide](https://grpc.io/docs/languages/python/quickstart/) for combining Python and gRPC as well as an [Introduction Tutorial](https://grpc.io/docs/languages/python/basics/) are provided online.

### Nomenclature
- ibaHD-API refers to the gRPC based API which comes with the ibaHD-Server.
- ibahd-api (Python package) refers to this Python package which simplifies the functionality of the ibaHD-API and extends it by useful helper functions.

## 2. Setup

### 2.1 Activate ibaHD-API
First, the API needs to be activated by following the corresponding [Quick Start Guide](https://github.com/iba-ag/ibaHD-API-Sample-Clients?tab=readme-ov-file#quick-start).

### 2.2 Setting up the example HD store
An example HD store backup (*ibahd-api_Backup*) for running this Jupyter Notebook can be found on [GitHub](https://github.com/iba-ag/ibaHD-API-Sample-Clients/tree/master/python). Follow the given steps below to mount this small example HD store in your ibaHD Manager:
1. Unzip the backup file.
2. Make sure to have ibaHD-Server with a sufficient version installed (see `Changelog.md` for compatibility overview).
3. Start ibaHD Manager and navigate to *Backup -> Mount -> Mount backups...*
4. Select the path of the folder where the unzipped backup HD store is located in (e.g. C:/Users/.../Downloads/).
5. Click Search.
7. Activate check-box 'Mount' and click *Next* and *OK* in the end. It is not necessary to change any other configuration.

The HD store should be listed in the table of "Mounted backups" under *Backups* now. Use its Name (*ibahd-api\_Backup*) in this notebook whenever the variable *hd\_store\_name* needs to be set.

### 2.3 Installation of the ibahd-api Python package
First of all, a Python environment needs to be created. It is recommended to use conda, however, feel free to use whatever environment you are used to (e.g. venv). Make sure to use a stable Python version<sup>1</sup> as a base.

To run this Jupyter Notebook, install the ibahd-api Python package with one of the following commands using the Python package installer `pip`. All necessary dependencies will be installed automatically.
```sh
> python -m pip install ibahd-api[tutorial]
```
or
```sh
> pip install ibahd-api[tutorial]
```

<sup>1</sup>: See [status of Python versions](https://devguide.python.org/versions/#supported-versions) for a list of supported versions.

### 2.4 Test for successfull installation
If the package installation was successfull you should be able to run the following code cell without getting any error message.

In [ ]:
import grpc  # for configuration of gRPC parameters when initializing the client
from pathlib import Path  # use for paths to avoid issues with different separators on Windows and Linux 
from ibahd_api import utils  # import the helper functions which extend the ibaHD-API functions
from ibahd_api import ibahd_api_pb2  # import the functions infered from ProtoBuf, contains request and response formats
from ibahd_api import ibahd_api_pb2_grpc  # import the basic functions of the ibahd-api Python package, contains the function calls to the ibaHD-Server

## 3. Application of the ibahd-api Python package
This chapter demonstrates the basic functions of the API paradigmatic on the provided example HD store. A proper parametrization enables the creation of a sample client which is connected with an ibaHD-Server. In addition, some commonly used functions are explained in more detail.

### 3.1 Parametrization for connection with ibaHD-Server
At the beginning, the ibaHD-Server connection parameters need to be set properly. For the parametrization two different ways are possible:
1. Use the \'quick connect\' information retrieved from ibaHD Manager to set the parameters easier.
    - In ibaHD Manager: *ibaHD-API -> Get API key... -> Copy quick connect info to clipboard*
    <p align="left"> <img width=600 style="padding-left: 100px" src="../assets/get_quick_connect.png"> </p>
2. Advanced: Set the connection parameters manually. The created certificate (see [Quick Start Guide](https://github.com/iba-ag/ibaHD-API-Sample-Clients?tab=readme-ov-file#quick-start)) needs to be stored locally and only its path is used here.
    - Reminder: The certificate needs to be stored in *.pem* format.

It is strongly recommended to use the first option, given its simplicity. However, both ways result in the exact same parametrization.   
After setting the connection parameters for the first time, you only need to change it when accessing an ibaHD-Server on a different computer. No adjustments are needed for multiple HD stores on the same computer.

In [ ]:
# Option 1: use 'quick connect info' to retrieve all necessary parameters at once
from ibahd_api import utils

quick_connect = Quick_Connect  # TODO: insert your copied 'quick connect info' here...

# Extract needed parameters from quick connect information
certificate, api_key, hostname, port = utils.get_api_params(quick_connect)

In [ ]:
# Option 2 (advanced): set parameters manually
path_certificate = Path(r'C:/Users/<your_username>/.../certificate_file.pem')  # path to local stored server TLS certificate file (.pem format)  # USER_TASK
certificate = open(path_certificate, 'rb').read()  # read exported (PEM formatted) server TLS certificate from disk
api_key = 'your_api_key'  # USER_TASK
hostname = 'name_of_your_server'  # USER_TASK
port = 9003  # usually no change required (default: 9003)  # USER_TASK

### 3.2 Functions of the ibaHD-API
This chapter shows the practical usage of the ibahd-api Python package and the ibaHD-API functions.
First, a gRPC client is created to access a HD store. Then, some commonly used functions are used for demonstration purposes. 

#### 3.2.1 Initialization of ibaHD-API client
Creates a gRPC client by using the previously set connection parameters. Again there are two ways to create the client:
1. Use the helper function provided by the Python package. This is recommended as usually no changes to the option parameters are required.
2. Advanced: Parametrize and create the client manually, which allows the customization of the channel options (if desired).

In [ ]:
# Option 1: use the provided helper function to create a gRPC client
from ibahd_api import utils

client = utils.create_client(certificate, api_key, hostname, port)

In [ ]:
# Option 2 (advanced): parametrize and create the gRPC client manually, add/adjust channel options if desired
import grpc
from ibahd_api import utils

# Combine certificate and api_key to full credentials used for authentification
tls_credentials = grpc.ssl_channel_credentials(certificate)
api_key_credentials  = grpc.metadata_call_credentials(utils.ApiKeyCallCredentials(api_key))  # apply api_key to every request made by the client
combined_credentials = grpc.composite_channel_credentials(tls_credentials, api_key_credentials)

# gRPC channel setup to connect to ibaHD-API endpoint in ibaHD-Server
endpoint = f'{hostname}:{port}'
options = [('grpc.max_receive_message_length', 2147483647)]  # increasing default message size (~4MB) recommended (c int32 max = 2147483647)

# Open gRPC channel with previous defined server
channel = grpc.secure_channel(endpoint, combined_credentials, options=options)

# Instantiate ibaHD-API client on the gRPC channel
client = ibahd_api_pb2_grpc.HdApiServiceStub(channel)

#### 3.2.2 Exemplified function calls
This section contains one code cell for each function of the ibaHD-API. Usually, the cells are divided into the following three parts:
- Parametrization
- API call
- Visualization

Descriptions of the function parameters, their types, and function outputs can be found in the proto-File<sup>1</sup> and the *ibaHD Server API* manual<sup>2</sup>.
   
<sup>1</sup>: By default stored at `C:\Program Files\iba\ibaHD-Server\ibaHD-API\ibaHD-API.proto`.   
<sup>2</sup>: Can be downloaded from the [homepage](https://www.iba-ag.com/de/downloads) -> Search for 'API' (Login necessary)

##### GetHDStores
Retrieve available HD stores from the connected ibaHD-Server. The provided HD store backup should be displayed in the list as well.

In [ ]:
# API call
request = ibahd_api_pb2.GetHdStoresRequest()
response = client.GetHdStores(request)

# Visualization
print(response)

##### GetHdStoreSchema
Retrieve the channel hierarchy for the provided HD store backup.

In [ ]:
# Parametrization
hd_store_name = 'ibahd-api_Backup'  # HD store for getting channel information
sort_by = 1  # 1: sort by module
info_fields = True  # if optional info fields for time or event channels should be requested

# API call
request = ibahd_api_pb2.GetHdStoreSchemaRequest(hd_store_name=hd_store_name,
                                                sort_by=sort_by,
                                                info_fields=info_fields)
response = client.GetHdStoreSchema(request)

# Visualization
print(response)

##### GetRawChannelData
Get raw data from the defined channel(s) in the defined time range. The retrieved data is plotted afterwards (data from n messages concatenated).

**Notes:**   
- The module utils contains some useful helper functions for converting between unix timestamps, datetime representations (strings), and C#/.NET ticks.
- The timestamps used by the ibaHD-API are on microsecond level, so it is necessary to multiply the unix timestamps with 1e6.
- Set datetimes and timestamps with **UTC timezone**.

In [ ]:
import matplotlib.pyplot as plt
# Helper functions to convert datetime string into unix timestamp (float) and vice versa
from ibahd_api.utils import convert_datetime_to_timestamp, convert_timestamp_to_datetime

# Parametrization
time_range_from = convert_datetime_to_timestamp('2025-7-2 15:30:0.0')  # start point in time for data retrieval (UNIX time in ms)
time_range_to = convert_datetime_to_timestamp('2025-7-2 15:45:0.0')  # end point in time for data retrieval (UNIX time in ms)
channel_ids = ['ibahd-api_Backup\\[0:0]']  # channel to retrieve data from
max_sample_count_per_message = 50  # number of datapoints per message
add_extra_sample_out_of_time_range = False  # get optional extra sample before and after time range

# API call
request = ibahd_api_pb2.GetRawChannelDataRequest(time_range_from=int(time_range_from*1e6),
                                                 time_range_to=int(time_range_to*1e6),
                                                 channel_ids=channel_ids,
                                                 max_sample_count_per_message=max_sample_count_per_message,
                                                 add_extra_sample_out_of_time_range=add_extra_sample_out_of_time_range)
response = client.GetRawChannelData(request)

# Visualization
nMessages = None  # how many data messages should be processed (set low for a test run)
nLabel = 20  # how many datetime labels should be displayed on the x axis
data = []
xticks, xlabel = [0], []

for i, chunk in enumerate(response):
    if nMessages and i == nMessages: break
    if i==0: xlabel.append(convert_timestamp_to_datetime(chunk.start_timestamp/1e6)[:-8])
    xticks.append(xticks[-1] + len(chunk.float_values))
    xlabel.append(convert_timestamp_to_datetime(chunk.start_timestamp/1e6 + chunk.step/1e6 * len(chunk.float_values))[:-8])
    data.extend(chunk.float_values)  # put data packages back together
xticks = xticks[::max(1,len(xticks)//nLabel)]  # show only some of the time points
xlabel = xlabel[::max(1,len(xlabel)//nLabel)]  # show only some of the time points
plt.figure(figsize=(22,2))
plt.plot(data)
plt.xticks(xticks, xlabel, rotation=90)
plt.show()
plt.close()

##### GetAggregatedChannelData
Get aggregated data from the defined channel(s) in the defined time range by applying the defined aggregation types.

In [ ]:
import matplotlib.pyplot as plt
# Helper functions to convert datetime string into unix timestamp (float) and vice versa
from ibahd_api.utils import convert_datetime_to_timestamp, convert_timestamp_to_datetime

# Parametrization
time_range_from = convert_datetime_to_timestamp('2025-7-2 15:30:0.0')  # start point in time for aggregated data retrieval (UNIX time in ms)
time_range_to = convert_datetime_to_timestamp('2025-7-2 15:45:0.0')  # end point in time for aggregated data retrieval (UNIX time in ms)
channel_ids = ['ibahd-api_Backup\\[0:0]']  # channel to retrieve aggregated data from
sample_count = 100  # how many datapoints should be retrieved (after aggregation) (set low for a test run)
min_aggregation, max_aggregation, avg_aggregation = True, True, True  # which aggregation types should be applied (multiple possible)
aggregation_algorithm_type = ibahd_api_pb2.AGGR_ALGO_TYPE_LINEAR_INTERPOLATION_DOWNSAMPLING  # which interpolation algorithm should be used
add_extra_sample_out_of_time_range = False

# API call
request = ibahd_api_pb2.GetAggregatedChannelDataRequest(time_range_from=int(time_range_from*1e6),
                                                        time_range_to=int(time_range_to*1e6),
                                                        channel_ids=channel_ids,
                                                        sample_count=sample_count,
                                                        min_aggregation=min_aggregation,
                                                        max_aggregation=max_aggregation,
                                                        avg_aggregation=avg_aggregation,
                                                        aggregation_algorithm_type=aggregation_algorithm_type,
                                                        add_extra_sample_out_of_time_range=add_extra_sample_out_of_time_range)
response = client.GetAggregatedChannelData(request)

# Visualization
nLabel = 20  # how many datetime labels should be displayed on the x axis
data = response.aggregated_channels[0].float_values.avg_values
xlabel = [convert_timestamp_to_datetime(e/1e6)[:-8] for e in response.timestamps]
xticks = list(range(len(xlabel)))
xticks = xticks[::max(1,len(xticks)//nLabel)]  # show only some of the time points
xlabel = xlabel[::max(1,len(xlabel)//nLabel)]  # show only some of the time points
plt.figure(figsize=(22,2))
plt.plot(data)
plt.xticks(xticks, xlabel, rotation=90)
plt.show()
plt.close()

##### GetHdTimePeriodStoreSchema
Retrieve information about the time period store (like column names).

In [ ]:
# Parametrization
hd_store_name = 'ibahd-api_Backup'  # HD store for getting channel information
time_period_store_name = 'TP_Store_1'  # Time Period store name
include_standard_fields = True  # if standard fields should be included in addition to the user defined info fields

# API call
request = ibahd_api_pb2.GetHdTimePeriodStoreSchemaRequest(hd_store_name=hd_store_name,
                                                          time_period_store_name=time_period_store_name,
                                                          include_standard_fields=include_standard_fields)
response = client.GetHdTimePeriodStoreSchema(request)

# Visualization
print(f'Standard fields:\n{[e.field_name for e in response.info_field_definitions if e.field_name[0] == "_"]}')
print(f'User defined info fields:\n{[e.field_name for e in response.info_field_definitions if e.field_name == e.display_name]}')

##### GetHdTimePeriodData
Retrieve time periods from the time period store.
For visualization purposes, the start and end time of the time periods is used with GetRawChannelData (or GetAggregatedChannelData) to display the time period data.

In [ ]:
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
# Helper functions to convert datetime string into unix timestamp (float) and vice versa
from ibahd_api.utils import convert_datetime_to_timestamp, convert_timestamp_to_datetime

# Parametrization
hd_store_name = 'ibahd-api_Backup'  # HD store for getting channel information
time_period_store_name = 'TP_Store_1'  # time period store name to retrieve time periods from
time_range_from = convert_datetime_to_timestamp('2025-7-2 15:30:0.0')  # start point in time for data retrieval (UNIX time in ms)
time_range_to = convert_datetime_to_timestamp('2025-7-2 15:45:0.0')  # end point in time for data retrieval (UNIX time in ms)
_show_columns = ['Counter_Sinus']
filter = ibahd_api_pb2.ColumnFilter(info_field_names=_show_columns)  # defines which of the user created info fields should be returned (standard fields always included)
query_mode = ibahd_api_pb2.QueryMode(is_start_time_in_time_range=True,
                                     is_end_time_in_time_range=False,
                                     include_open=False,
                                     column_filter_active=True)  # Query mode to define if start time and end time is included in query time range or not and if below defined filter should be applied
max_sample_count_per_message = 100  # number of datapoints per message

# API call
request = ibahd_api_pb2.GetHdTimePeriodDataRequest(hd_store_name=hd_store_name,
                                                   time_period_store_name=time_period_store_name,
                                                   time_range_from=int(time_range_from*1e6),
                                                   time_range_to=int(time_range_to*1e6),
                                                   query_mode=query_mode,
                                                   filter=filter,
                                                   max_sample_count_per_message=max_sample_count_per_message)
response = client.GetHdTimePeriodData(request)

# Visualization
time_periods = [e for chunk in response for e in chunk.time_period_data][::-1]  # get all segments from all retrieved packages in response (flattened)

# Show TimePeriod informations as table
table = [(e.id, e.name, convert_timestamp_to_datetime(e.start_time/1e6), convert_timestamp_to_datetime(e.end_time/1e6), e.stop_trigger, e.metadata_id, e.double_fields[0].value) for e in time_periods]
print(tabulate(table, headers=['ID','Name','StartTime','EndTime','StopTrigger','MetaDataID',time_periods[0].double_fields[0].name]))

# Get the corresponding data for the retrieved time periods by using GetRawChannelData (alternative: GetAggregatedChannelData) and plot them
# For detailed explanation see the cells above
def put_data_together(start_time, res):  # put multiple data packages (gRPC messages) together and extract the step size
    first_msg = next(res)
    first_vals, step = list(first_msg.float_values), first_msg.step
    vals = list(zip(first_vals, range(start_time, start_time+len(first_vals)*step, step)))
    for chunk in res:
        cur_start_time = start_time + (len(vals)+1) * step
        cur_vals = chunk.float_values
        vals.extend(zip(cur_vals, range(cur_start_time, cur_start_time+len(cur_vals)*step, step)))
    return vals

xticks = []
data_combined = []
time_periods = time_periods[:10]  # show only first 10 time periods
for tp in time_periods:
    request = ibahd_api_pb2.GetRawChannelDataRequest(time_range_from=tp.start_time,
                                                     time_range_to=tp.end_time,
                                                     channel_ids=['ibahd-api_Backup\\[0:0]'],
                                                     max_sample_count_per_message=10000,
                                                     add_extra_sample_out_of_time_range=False)
    response = client.GetRawChannelData(request)
    values = put_data_together(tp.start_time, response)
    data_combined.extend(values)
    xticks.extend([tp.start_time, tp.end_time])

plt.figure(figsize=(22,2))
y, x = zip(*data_combined)
plt.scatter(x, y, s=0.8)
for xc in xticks:
    plt.axvline(x=xc, linestyle='dotted')
xlabel = [convert_timestamp_to_datetime(e/1e6)[:-8] for e in xticks]
plt.xticks(xticks, xlabel, rotation=90)
plt.show()

In [ ]:
# ------------------------------------------------------------
# more functions will be added in the future...

In [ ]:
# Clean channel shutdown
channel.close()